# Test: what is the "false positive" rate for young stellar populations?
---
We want to be sure that we can accurately detect young stellar populations using ppxf. 
However, ppxf may return an SFH with nonzero weights in the young SSP templates when none exist in the input. 
* What are typical template weights of the young templates when the input has no young component?
* Do these weights change as a function of S/N? i.e., are the erroneous weights higher when the noise is higher? Or are they roughly the same, regardless?
* What is the *total* erroneous stellar mass in the young bins?
* When regularisation is used, does the problem persist?

Ideas:
* Take one of Phil's SFHs with no young stellar component & run tests. Galaxy 0024 is a good candidate.
* Then artificially add a young stellar population & repeat. What is the minimum mass we can have in the young component that can reliably be detected with ppxf? i.e., if ppxf says there is ~10^8 Msun in young templates when the input only contains old templates, then that means we cannot reliably detect YSPs with this (relative) mass.

In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [3]:
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing
import pandas as pd

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates
from ppxftests.mockspec import load_sfh, create_mock_spectrum, calculate_mw_age
from ppxftests.ppxf_plot import plot_sfh_mass_weighted

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [4]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


## Does S/N affect the spurious nonzero weights in the lowest age bins?
---
Using as input an SFH containing no young stellar populations, do our analysis several times, varying the input S/N.


In [5]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
sigma_star_kms = 250
z = 0.01

niters = 100
nthreads = 20

# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)


### Old stellar population only

In [15]:
###########################################################################
# Generate the input SFH
###########################################################################
sfh_mw_original = load_sfh(24, plotit=True)
sfh_mw_1D_original = np.nansum(sfh_mw_original, axis=0)
M_tot_original = np.nansum(sfh_mw_original)

/home/u5708159/python/Modules/ppxftests/ppxf_plot.py:31: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 3.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
###########################################################################
# Run multiple times with varying S/N; plot the resulting SFHs 
###########################################################################
SNR_vals = np.logspace(0, 5, 11)
age_idx = np.nanargmin(np.abs(ages - 1e9))
df = pd.DataFrame()

for SNR in SNR_vals:
    ###########################################################################
    # Generate spectrum
    ###########################################################################
    spec_old, spec_old_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_original,
        agn_continuum=False,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)
        
    ###########################################################################
    # Run MC simulations with ppxf
    ###########################################################################
    # Input arguments
    seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
    args_list = [[s, spec_old, spec_old_err, lambda_vals_A] for s in seeds]

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_list_old = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Elapsed time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Compute mean quantities from the MC runs
    ###########################################################################
    sfh_fit_mw_list = [pp.weights_mass_weighted for pp in pp_list_old]
    sfh_fit_lw_list = [pp.weights_light_weighted for pp in pp_list_old]
    sfh_fit_mw_1D_list = [np.nansum(pp.weights_mass_weighted, axis=0) for pp in pp_list_old]
    sfh_fit_lw_1D_list = [np.nansum(pp.weights_light_weighted, axis=0) for pp in pp_list_old]
    
    M_tot_list = [np.nansum(pp.weights_mass_weighted) for pp in pp_list_old]
    M_tot_young_list = [np.nansum(pp.weights_mass_weighted[:, :age_idx]) for pp in pp_list_old]

    # Compute the mean SFH 
    sfh_fit_mw_mean = np.nansum(np.array(sfh_fit_mw_list), axis=0) / len(sfh_fit_mw_list)
    sfh_fit_lw_mean = np.nansum(np.array(sfh_fit_lw_list), axis=0) / len(sfh_fit_lw_list)
    sfh_fit_mw_1D_mean = np.nansum(sfh_fit_mw_mean, axis=0)
    sfh_fit_lw_1D_mean = np.nansum(sfh_fit_lw_mean, axis=0)
    
    # Compute the mean mass 
    M_tot_mean = np.nanmean(M_tot_list)
    M_tot_err = np.nanstd(M_tot_list)
    M_tot_young_mean = np.nanmean(M_tot_young_list)
    M_tot_young_err = np.nanstd(M_tot_young_list)

    # sfh_fit_mw_1D_regul = np.nansum(pp_regul.weights_mass_weighted, axis=0)
    # sfh_fit_lw_1D_regul = np.nansum(pp_regul.weights_light_weighted, axis=0)
    
    ###########################################################################
    # Compute the total mass, plus the total mass < 1 Gyr
    ###########################################################################
    M_tot = np.nansum(sfh_fit_mw_1D_mean)
    M_tot_young = np.nansum(sfh_fit_mw_1D_mean[:age_idx])
    
    ###########################################################################
    # Plot the mass-weighted weights, summed over the metallicity dimension
    ###########################################################################
    for log_scale in [True]:
        # Create new figure 
        fig = plt.figure(figsize=(13, 4))
        ax = fig.add_axes([0.1, 0.2, 0.7, 0.7])
        ax.set_title(f"Mass-weighted template weights (S/N = {SNR})")

        # Plot the SFHs from each ppxf run, plus the "truth" SFH
        ax.fill_between(range(N_ages), sfh_mw_1D_original, step="mid", alpha=1.0, color="cornflowerblue", label="Input SFH")
        for jj in range(niters):
            ax.step(range(N_ages), sfh_fit_mw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
        ax.step(range(N_ages), sfh_fit_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
        # ax.step(range(N_ages), sfh_fit_mw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")

        # Decorations 
        ax.set_xticks(range(N_ages))
        ax.set_xlabel("Age (Myr)")
        ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
        ax.autoscale(axis="x", enable=True, tight=True)
        ax.set_ylim([1, None])
        ax.set_ylabel(r"Template weight ($\rm M_\odot$)")
        ax.legend(fontsize="x-small", loc="center left", bbox_to_anchor=(1.01, 0.5))
        ax.set_xlabel("Age (Myr)")
        ax.set_yscale("log") if log_scale else None
        
    ###########################################################################
    # Append information to DataFrame
    ###########################################################################
    df = df.append({
            "SNR": SNR,
            "Total mass": M_tot,
            "Total mass error": M_tot_err,
            "Total mass < 1 Gyr": M_tot_young,
            "Total mass < 1 Gyr error": M_tot_young_err
        }, ignore_index=True)
    

Running ppxf on 20 threads...



Elapsed time in ppxf: 123.94 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 148.91 s


...


Elapsed time in ppxf: 163.67 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 176.62 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
###########################################################################
# Plot the total mass < 1 Gyr vs. S/N ratio
###########################################################################
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.15, 0.5, 0.75])
ax.errorbar(x=df["SNR"].values, 
            y=df["Total mass"].values, 
            yerr=df["Total mass error"].values,
            linestyle="none", marker="D", label=r"MC $M_{\rm tot}$", zorder=999)
ax.errorbar(x=df["SNR"].values, 
            y=df["Total mass < 1 Gyr"].values, 
            yerr=df["Total mass < 1 Gyr error"].values,
            linestyle="none", marker="D", label=r"MC $M_{\rm tot}\, (\rm < 1 \, Gyr)$", zorder=999)
ax.axhline(M_tot_original, color="grey", label="Total mass")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_xlabel("S/N ratio")
ax.set_ylabel(r"Stellar mass ($\rm M_\odot$)")
ax.set_title("No young stellar component")
ax.grid()

# Plot again, but as a fraction of the total mass
df["Young mass fraction"] = df["Total mass < 1 Gyr"] / df["Total mass"]
df["Young mass fraction error"] = df["Young mass fraction"] * np.sqrt((df["Total mass < 1 Gyr error"] / df["Total mass < 1 Gyr"])**2 + 
                                                                      (df["Total mass error"] / df["Total mass"])**2)
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.15, 0.5, 0.75])
ax.errorbar(x=df["SNR"].values, 
            y=df["Young mass fraction"].values * 100, 
            yerr=df["Young mass fraction error"].values * 100,
            linestyle="none", marker="D", label=r"MC $M_{\rm tot}\, (\rm < 1 \, Gyr)$", zorder=999)
ax.axhline(100, color="grey", label="Total mass")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_xlabel("S/N ratio")
ax.set_ylabel(r"Mass fraction of young stellar population (%)")
ax.set_title("No young stellar component")
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Remarks 
---
The total mass in the spurious "young" stellar population decreases as the S/N increases, and reaches a plateau at approximately $\sim 10^6 \,\rm M_\odot$, corresponding to $< 0.001\%$ of the total mass.
At S/N ratios below 100, however, the mass in the spurious young stellar population increases to a few $10^7 \,\rm M_\odot$, representing $>0.01\%$ of the total mass. In the worst case, when the S/N = 1, this fraction increases to approximately $0.3\%$.

## Test: add a young stellar component

In [18]:
###########################################################################
# Generate the input SFH
###########################################################################
sfh_mw_original = load_sfh(24, plotit=True)
sfh_mw_1D_original = np.nansum(sfh_mw_original, axis=0)

# Add a young component, plot to check
sfh_mw_young = np.copy(sfh_mw_original)
sfh_mw_young[0, 5] = 1e7
sfh_mw_1D_young = np.nansum(sfh_mw_young, axis=0)

plot_sfh_mass_weighted(sfh_mw_young, ages, metallicities)
plt.gcf().get_axes()[0].set_title("Young stellar population added")

M_tot_original = np.nansum(sfh_mw_young)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
###########################################################################
# Run multiple times with varying S/N; plot the resulting SFHs 
###########################################################################
SNR_vals = np.logspace(0, 5, 6)
age_idx = np.nanargmin(np.abs(ages - 1e9))
df = pd.DataFrame()

for SNR in SNR_vals:
    ###########################################################################
    # Generate spectrum
    ###########################################################################
    spec_young, spec_young_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_young,
        agn_continuum=False,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)
        
    ###########################################################################
    # Run MC simulations with ppxf
    ###########################################################################
    # Input arguments
    seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
    args_list = [[s, spec_young, spec_young_err, lambda_vals_A] for s in seeds]

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_list_young = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Elapsed time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Compute mean quantities from the MC runs
    ###########################################################################
    sfh_fit_mw_list = [pp.weights_mass_weighted for pp in pp_list_young]
    sfh_fit_lw_list = [pp.weights_light_weighted for pp in pp_list_young]
    sfh_fit_mw_1D_list = [np.nansum(pp.weights_mass_weighted, axis=0) for pp in pp_list_young]
    sfh_fit_lw_1D_list = [np.nansum(pp.weights_light_weighted, axis=0) for pp in pp_list_young]
    
    M_tot_list = [np.nansum(pp.weights_mass_weighted) for pp in pp_list_young]
    M_tot_young_list = [np.nansum(pp.weights_mass_weighted[:, :age_idx]) for pp in pp_list_young]

    # Compute the mean SFH 
    sfh_fit_mw_mean = np.nansum(np.array(sfh_fit_mw_list), axis=0) / len(sfh_fit_mw_list)
    sfh_fit_lw_mean = np.nansum(np.array(sfh_fit_lw_list), axis=0) / len(sfh_fit_lw_list)
    sfh_fit_mw_1D_mean = np.nansum(sfh_fit_mw_mean, axis=0)
    sfh_fit_lw_1D_mean = np.nansum(sfh_fit_lw_mean, axis=0)
    
    # Compute the mean mass 
    M_tot_mean = np.nanmean(M_tot_list)
    M_tot_err = np.nanstd(M_tot_list)
    M_tot_young_mean = np.nanmean(M_tot_young_list)
    M_tot_young_err = np.nanstd(M_tot_young_list)

    # sfh_fit_mw_1D_regul = np.nansum(pp_regul.weights_mass_weighted, axis=0)
    # sfh_fit_lw_1D_regul = np.nansum(pp_regul.weights_light_weighted, axis=0)
    
    ###########################################################################
    # Compute the total mass, plus the total mass < 1 Gyr
    ###########################################################################
    age_idx = np.nanargmin(np.abs(ages - 1e9))
    M_tot = np.nansum(sfh_fit_mw_1D_mean)
    M_tot_young = np.nansum(sfh_fit_mw_1D_mean[:age_idx])
    
    ###########################################################################
    # Plot the mass-weighted weights, summed over the metallicity dimension
    ###########################################################################
    for log_scale in [True]:
        # Create new figure 
        fig = plt.figure(figsize=(13, 4))
        ax = fig.add_axes([0.1, 0.2, 0.7, 0.7])
        ax.set_title(f"Mass-weighted template weights (S/N = {SNR}) - with young stellar population added")

        # Plot the SFHs from each ppxf run, plus the "truth" SFH
        ax.fill_between(range(N_ages), sfh_mw_1D_young, step="mid", alpha=1.0, color="cornflowerblue", label="Input SFH")
        for jj in range(niters):
            ax.step(range(N_ages), sfh_fit_mw_1D_list[jj], color="pink", alpha=0.2, where="mid", linewidth=0.25, label="ppxf fits (MC simluations)" if jj == 0 else None)
        ax.step(range(N_ages), sfh_fit_mw_1D_mean, color="red", where="mid", label="Mean ppxf fit (MC simulations)")
        # ax.step(range(N_ages), sfh_fit_mw_1D_regul, color="lightgreen", where="mid", label="ppxf fit (regularised)")

        # Decorations 
        ax.set_xticks(range(N_ages))
        ax.set_xlabel("Age (Myr)")
        ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
        ax.autoscale(axis="x", enable=True, tight=True)
        ax.set_ylim([1, None])
        ax.set_ylabel(r"Template weight ($\rm M_\odot$)")
        ax.legend(fontsize="x-small", loc="center left", bbox_to_anchor=(1.01, 0.5))
        ax.set_xlabel("Age (Myr)")
        ax.set_yscale("log") if log_scale else None
        
    ###########################################################################
    # Append information to DataFrame
    ###########################################################################
    df = df.append({
            "SNR": SNR,
            "Total mass": M_tot,
            "Total mass error": M_tot_err,
            "Total mass < 1 Gyr": M_tot_young,
            "Total mass < 1 Gyr error": M_tot_young_err
        }, ignore_index=True)
    

Running ppxf on 20 threads...



Elapsed time in ppxf: 149.10 s


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...


...


Elapsed time in ppxf: 176.27 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Running ppxf on 20 threads...



Elapsed time in ppxf: 173.22 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
###########################################################################
# Plot the total mass < 1 Gyr vs. S/N ratio
###########################################################################
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.15, 0.5, 0.75])
ax.errorbar(x=df["SNR"].values, 
            y=df["Total mass"].values, 
            yerr=df["Total mass error"].values,
            linestyle="none", marker="D", label=r"MC $M_{\rm tot}$", zorder=999)
ax.errorbar(x=df["SNR"].values, 
            y=df["Total mass < 1 Gyr"].values, 
            yerr=df["Total mass < 1 Gyr error"].values,
            linestyle="none", marker="D", label=r"MC $M_{\rm tot}\, (\rm < 1 \, Gyr)$", zorder=999)
ax.axhline(M_tot_original, color="grey", label="Total mass")
ax.axhline(M_tot_young, color="grey", label="Young component")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_xlabel("S/N ratio")
ax.set_ylabel(r"Stellar mass ($\rm M_\odot$)")
ax.set_title("Young stellar component")
ax.grid()

# Plot again, but as a fraction of the total mass
df["Young mass fraction"] = df["Total mass < 1 Gyr"] / df["Total mass"]
df["Young mass fraction error"] = df["Young mass fraction"] * np.sqrt((df["Total mass < 1 Gyr error"] / df["Total mass < 1 Gyr"])**2 + 
                                                                      (df["Total mass error"] / df["Total mass"])**2)
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.15, 0.5, 0.75])
ax.errorbar(x=df["SNR"].values, 
            y=df["Young mass fraction"].values * 100, 
            yerr=df["Young mass fraction error"].values * 100,
            linestyle="none", marker="D", label=r"MC $M_{\rm tot}\, (\rm < 1 \, Gyr)$", zorder=999)
ax.axhline(100, color="grey", label="Total mass")
ax.axhline(M_tot_young / M_tot * 100, color="black", label="Young component")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend(bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.set_xlabel("S/N ratio")
ax.set_ylabel(r"Mass fraction of young stellar population (%)")
ax.set_title("Young stellar component")
ax.grid()

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …